# <center> EVSC 601 Interactive Learning: Effects, Hypothesis Testing and More

This notebook lets you explore what an *effect* means in hypothesis testing by simulating many experiments, as well as exploring more basic concepts in statistics (and probability) including the meaning of a sampling distribution, and *p-values* with respect to correlation analysis.
<br>
<br>
A core theme of this notebook is to relate learning directly to the reading of Furbish, D. and Schmeeckle, M. (2021) [https://cdn.vanderbilt.edu/t2-my/my-prd/wp-content/uploads/sites/978/2013/06/21st-Century-Statistics.pdf], and to place front and center that statistics is about understanding variability through models and repeated reasoning, not about executing rote procedures once. Statistics (and probability) is more than a procedure or algorithm to memorize and use because others are using it.
<br>
<br>
The notebook is accessible on a GitHub Pages site I built, which permits one to run the notebook through a web browser without the need to have Python installed [https://smchartrand.github.io/JupyterLite-Deployment/lab/index.html]

## Definitions: To make sure we are all oriented in the same context, the following provides some important definitions to the concepts explored below.
<br>

- *Effect* --> (1) An effect is something that changes the distribution of possible experimental outcomes across repetitions — not something that appears magically when p < 0.05 OR (2) An effect is a shift in the data-generating process that moves the sampling distribution away from what we would expect under pure randomness -- hypothesis testing asks whether your observed result (statistic calculated from your population sample) is more consistent with randomness or some specific effect, physical or otherwise. In the calculations below, the *effect* is the difference between *mu* and *mu0*, i.e. the difference between the alternative mean and the null mean values.
<br>

- *Sample Size* --> How many or the number of things that comprise your sample. The sample size is constant across all simulated experiments. For example, if you go into the field to collect data, the sample size, in general, reflects how many measurements you may make at every location in your sample plan. A measurement may be the length of a leaf, which you measure 30 times at a given location.
<br>

- *Number of Simulations* --> The number of simulated experiments OR the number of samples drawn from a population (all have the same dimensions). For example, if you go into the field to collect data, the number of simulations, in general, reflects how many locations are used to make your measurements of interest. You may select 20 different plots at random to make your leaf length measurements. 
<br>

- *mu* --> mean (population parameter)
<br>

- *sigma* --> standard deviation (population parameter)
<br>

### What's up with sample statistics?
- In general, a sample statistic is any numerical quantity computed from the observed data alone (e.g. mean, std. deviation, etc.).
- Before you collect data, you don’t know which sample you’ll get, so you don’t know the value of the statistic.
- Therefore, a statistic is a *random* variable before observation, and a number after observation.
<br>

### Instructions to use the notebook
- Run each cell in the notebook by clicking on the right arrow button (e.g. &#9658;) at the top. When you click the &#9658;, the notebook will advance to the next cell. Keep advancing until you have plotted the results and the slider controls are available for use. Once you have run all the cells and the slider controls are available, change the parameters below by using the slider controls:
1. Set effect_size by changing the value of 'mu' for the "Alternative mean (μ)". Increase it to see stronger effects.
2. Increase `sample_size` to see how power changes
- Useful question to think about as you use the notebook:
1.  What changes in the sampling distribution as you modify parameter values?
2.  Under which condition do more extreme z-values occur?
3.  How are p-values affected? When do you observe a non-uniform distribution of p-values?
4.  Is the sample mean a property of the population (the thing we are drawing samples from), or a property of the experiment?

In [1]:
# Import the needed libraries
import numpy as np
from numpy.random import default_rng
from scipy.stats import norm
import matplotlib.pyplot as plt
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, IntSlider
import ipywidgets as widgets

rng = np.random.default_rng()

# Things needed for the simulation or the numerical experiment, depending on how you want to think about it
# Fixed population parameters (unknown in real life)
mu0 = 0.0             # simulated null mean
mu = 0.0              # simulated alternative mean (you can change below to explore effects)
sigma = 0.5           # simulated standard deviation
# -----------------------
# Parameters to explore
# -----------------------
sample_size = 30      # per group
n_sims = 1000         # number of simulated experiments
alpha = 0.05          # threshold

### <center> Use a normal distribution with specified mean and std deviation to produce a sampling distribution of means

In [2]:
# Generate a mean statistic for a numerical array produced by randomly sampling from a normal distribution. 

def sampling_and_power(mu=mu, sigma=sigma, sample_size=sample_size, n_sims=n_sims):
    
    means = []
    z_stats = []
    p_vals = []
    
    for _ in range(n_sims):
        
        # Create a psuedo-random array by sampling from a normal distribution
        sample = rng.normal(mu, sigma, sample_size)
        # Calculate the mean of the pseudo-random array
        xbar = sample.mean()
        # The test statistic measures extremeness relative to variability
        z = (xbar - mu0) / (sigma / np.sqrt(sample_size))
        # Calculate the theoretical p-value
        p = 2 * (1 - norm.cdf(abs(z)))
        # We could also calculate an empirical p-value
        # p_empirical = np.mean(abs(z_stats) >= abs(z_obs))
        
        # Store values
        means.append(xbar)
        z_stats.append(z)
        p_vals.append(p)

    p_vals = np.array(p_vals)
    # Calculate the statistical power of the simulations
    power = np.mean(p_vals < alpha)
    
    # Create the fgiure environment
    fig, axes = plt.subplots(1, 3, figsize=(15,4))
    
    # LEFT-HAND PLOT
    # Alternative hypothesis sample means histogram
    axes[0].hist(means, bins=50, density=True, color="coral", alpha=0.7, label="Observed sample means")
    # Null sampling distribution (theoretical)
    se = sigma / np.sqrt(sample_size)
    # Create a null distribution
    x = np.linspace(mu0 - 4*se, mu0 + 4*se, 400)
    null_pdf = norm.pdf(x, loc=mu0, scale=se)
    # Null distribution plot
    axes[0].plot(x, null_pdf, color='black', linewidth=2, label='Null distribution')
    axes[0].set_ylabel("Probability density\n(area = 1)")
    # True mean line (alternative)
    axes[0].axvline(mu, color='gray', linestyle='--', label='Alternative mean')
    axes[0].set_title("Sampling Distribution of the Mean")
    axes[0].legend(fontsize = 8)

    # Z-statistics historgram
    axes[1].hist(z_stats, bins=50, density=True, color="coral", alpha=0.7)
    axes[1].axvline(0, color='gray', linestyle='--')
    axes[1].set_title("Z-statistics (relative to H₀)")
    
    # P-values histogram
    axes[2].hist(p_vals, bins=50, density=True, color="coral", alpha=0.7)
    axes[2].axvline(0.05, color='gray', linestyle='--', label='α = 0.05')
    axes[2].set_title("P-values")
    axes[2].legend()
    # Shade the "significant" region
    axes[2].axvspan(0, 0.05, color='lightgray', alpha=0.3, label=f'Significant area (empirical power ~ {np.mean(p_vals<0.05):.2f})')

     # Power annotation
    fig.text(
        0.5, -0.05,
        f"Empirical power (P[p < 0.05 | H₁]): {power:.3f}",
        ha="center",
        fontsize=12
    )
    
    plt.tight_layout()
    plt.show()

### <center> Now an interactive plot environment is made with slider controls to explore the sampling distribution problem 

In [3]:
# Code to create the slider controls for mu, sigma, sample size and number of simulations (experiments)

mu_slider = widgets.FloatSlider(
    value=0.0, min=-0.5, max=0.5, step=0.05,
    description="Alternative mean (μ)",
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

sigma_slider = widgets.FloatSlider(
    value=1.0, min=0.1, max=3.0, step=0.1,
    description="Noise (σ)",
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

n_slider = widgets.IntSlider(
    value=30, min=5, max=200, step=5,
    description="Sample size (n)",
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

sims_slider = widgets.IntSlider(
    value=1000, min=200, max=5000, step=200,
    description="Simulations",
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

ui = widgets.VBox([mu_slider, sigma_slider, n_slider, sims_slider])
out = widgets.Output()

def update_plot(change=None):
    with out:
        out.clear_output(wait=True)
        sampling_and_power(
            mu_slider.value,
            sigma_slider.value,
            n_slider.value,
            sims_slider.value
        )

for slider in [mu_slider, sigma_slider, n_slider, sims_slider]:
    slider.observe(update_plot, names='value')

display(ui, out)
update_plot()


Output()

### Question to relate the exercise to a tangible research design problem
- Let's imagine that you are going to develop a sampling plan (design an experiment) to measure some environmental variable. You have a finite amount of time to conduct your fieldwork and the work will be done in a remote location.
- When developing your sampling plan, in general, is it more useful to use a relatively large sample size at each sampling location (simulations in the above set-up), or is it more useful to use a relatively large number of sampling locations?
- What is the evidence to support your decision.
- In either case, what is a minimum number of sample size that you think must be collected at each sampling location to adequately describe the data generating process? 